In [1]:
import json

In [2]:
f = open("../task2/myfile.json", "r", encoding='utf-8')
sl = json.load(f)
f.close()

In [3]:
def PersonalRank(G, alpha, root, max_step):   
    weightMat = {x:0 for x in G.keys()}  
    weightMat[root] = 1  
    for _ in range(max_step):  
        weightMatCopy = {x:0 for x in G.keys()}   
        for node, neigh in G.items():  
            for j, wij in neigh.items(): 
#                 print(j)
#                 print(node)
                weightMatCopy[j]=weightMatCopy.get(j,0)+alpha * weightMat[node] * wij / sum(neigh.values())  
        weightMatCopy[root] += (1 - alpha)  
        weightMat = weightMatCopy  

#         print('iter:'  + str(k) + "\t")  
#         for key, value in rank.items():  
#             print("%s:%.3f, \t"%(key, value))  

    return weightMat  

This may sound weird, but PersonalRank, as defined above, converges at the second step with alpha = 0.85.

In [4]:
counter=0
for i in sl:
    counter+=1
    if counter>=10:
        break
    res=PersonalRank(sl, 0.85, i, 10)
    res2=PersonalRank(sl, 0.85, i, 2)
    print(max(abs(res2[i]-res[i]) for i in res))

0.0
0
0
0
0
0
0
0
0


In [5]:
len(sl)

1309

In [6]:
uniqueTokens=set()
for i in sl:
    uniqueTokens |= set(sl[i])

In [7]:
len(uniqueTokens)

19217

In [8]:
len(res)

20526

In [9]:
len(sl)+len(uniqueTokens)

20526

Therefore PersonalRank calculates weights for both emojis and words. We need to exclude (emoji, emoji) pairs.

In [13]:
weightDict=dict()
wordKey=set()
counter=0
for emoji in sl:
    counter+=1
    if counter%100==0:
        print("Processing "+str(counter))
    res=PersonalRank(sl, 0.85, emoji, 2)
    for word, weight in res.items():
        if weight==0 or emoji==word:
            continue
            
        if word not in wordKey:
            wordKey.add(word)
            weightDict[word]={emoji:weight}
        else:
            weightDict[word][emoji]=weightDict[word].get(emoji,0)+weight
    

Processing 100
Processing 200
Processing 300
Processing 400
Processing 500
Processing 600
Processing 700
Processing 800
Processing 900
Processing 1000
Processing 1100
Processing 1200
Processing 1300


In [14]:
# weightDict

In [15]:
max(len(weightDict[i]) for i in weightDict)

745

In [16]:
out_file = open("personalRankRaw.json", "w") 
json.dump(weightDict, out_file) 
out_file.close()

In [17]:
weightDictNormalized=weightDict.copy()

In [19]:
for i in weightDictNormalized:
    coe=sum(weightDictNormalized[i].values())
    for k, v in weightDictNormalized[i].items():
        weightDictNormalized[i][k]=v/coe

In [20]:
out_file = open("personalRankNormalized.json", "w") 
json.dump(weightDict, out_file) 
out_file.close()